In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [4]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])


Epoch 1/5
   1/1875 [..............................] - ETA: 0s - loss: 2.4032 - accuracy: 0.0938WARNING:tensorflow:From /home/mnpr_term/miniconda3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2219 - accuracy: 0.9347 - val_loss: 0.1191 - val_accuracy: 0.9644
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0979 - accuracy: 0.9698 - val_loss: 0.0771 - val_accuracy: 0.9748
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0683 - accuracy: 0.9782 - val_loss: 0.0695 - val_accuracy: 0.9782
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0519 - accuracy: 0.9826 - val_loss: 0.0608 - val_accuracy: 0.9814
Epoch 5/5
1875/1875 [===========================

### `%tensorboard --logdir logs/fit`

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [10]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [11]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

In [12]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [13]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.24778307974338531, Accuracy: 92.7683334350586, Test Loss: 0.11668650060892105, Test Accuracy: 96.45000457763672
Epoch 2, Loss: 0.10510851442813873, Accuracy: 96.81999969482422, Test Loss: 0.08729000389575958, Test Accuracy: 97.33000183105469
Epoch 3, Loss: 0.07297644019126892, Accuracy: 97.7683334350586, Test Loss: 0.07107486575841904, Test Accuracy: 97.83999633789062
Epoch 4, Loss: 0.05591828003525734, Accuracy: 98.288330078125, Test Loss: 0.07340776920318604, Test Accuracy: 97.61000061035156
Epoch 5, Loss: 0.04327007383108139, Accuracy: 98.71666717529297, Test Loss: 0.06331400573253632, Test Accuracy: 98.00999450683594


### `%tensorboard --logdir logs/gradient_tape`

***

In [ ]:
# Simple Linear Regression

